In [6]:
# Import libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import seaborn as sns
import random
import scipy
from scipy.stats import chi2

pd.set_option('display.max_columns', None)

In [3]:
# Read the data
df_clinical = pd.read_csv('data/pcawg_donor_clinical_August2016_v9.csv', sep='\t')

df_TGA = pd.read_excel('data/TCGA-CDR-SupplementalTableS1.xlsx')
df_TGA = df_TGA.rename(columns={'bcr_patient_barcode': 'submitted_donor_id'})

df_design = pd.read_csv('data/PCAWG_experiment-design.tsv', sep='\t')
df_design = df_design.rename(columns={'Sample Characteristic[individual]': 'icgc_donor_id'})

In [12]:
# Merge the three data frames
df = pd.merge(df_TGA, df_clinical, on="submitted_donor_id")
df = pd.merge(df, df_design, on='icgc_donor_id')

# Filter by cancer type
df=df[df['type']=='BLCA']

# Formatting the gender: FEMALE -> 0, MALE -> 1
df['gender'] = df['gender'].replace(to_replace='FEMALE', value=0)
df['gender'] = df['gender'].replace(to_replace='MALE', value=1)

# Removing NA's
df['OS.time'] = df['OS.time'].fillna(0) # 3 NA's
df['age_at_initial_pathologic_diagnosis'] = df['age_at_initial_pathologic_diagnosis'].fillna(0) # 4 NA's
# NA's: two ideas
#   1. mean
#   2. remove samples

In [14]:
# Reduced data frame with the desired parameters
cox_df=df[['age_at_initial_pathologic_diagnosis','OS.time','gender','OS']] 

# Fitting the model and printing the results
cph = CoxPHFitter()
cph.fit(cox_df, duration_col = 'OS.time', event_col='OS')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 27 total observations, 14 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 27
number of events observed = 13
   partial log-likelihood = -31.76
         time fit was run = 2023-01-19 11:49:48 UTC

---
                                      coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                   
age_at_initial_pathologic_diagnosis   0.08       1.08       0.03             0.02             0.14                 1.02                 1.15
gender                                0.08       1.09       0.62            -1.13             1.29                 0.32                 3.65

                                      cmp to    z    p   -log2(p)
covariate                                                        
age_at_initial_pathologic_diagnosis     0.00 2.65 0.01       6.96
gender                                  0.00 0.13 0.89       0.16
---
Concordance = 0.74
Partial AIC = 67.53
log-likelihood ratio test = 9.07 on 2 df
-log2(p) of ll-ratio test = 6.55